In [ ]:
!pip install pandas matplotlib numpy seaborn

In [ ]:
import pandas as pd
import numpy as np
# import matplotlib.plyplot as plt
import seaborn as sns
import re
from textblob import TextBlob
import warnings

In [ ]:
df = pd.read_csv("steam_store_data_2024.csv")
print(f"Shape: {df.shape}")

Shape: (86, 6)


In [ ]:
print(df.head())

                                               title  \
0                      Ori and the Will of the Wisps   
1  Flashing Lights - Police, Firefighting, Emerge...   
2                                         Thronefall   
3  DRAGON QUEST® XI S: Echoes of an Elusive Age™ ...   
4                                            UNDYING   

                                         description   price salePercentage  \
0  Play the critically acclaimed masterpiece. Emb...   $9.89           -67%   
1  Play solo or in up to 10-player multiplayer co...   $8.49           -66%   
2  A minimalist game about building and defending...   $5.24           -25%   
3  The Definitive Edition includes the critically...  $23.99           -40%   
4  As Anling’s zombie infection sets in, her days...  $13.99           -30%   

             recentReviews               allReviews  
0  Overwhelmingly Positive  Overwhelmingly Positive  
1            Very Positive            Very Positive  
2  Overwhelmingly Positive

In [ ]:
print(df.columns.tolist())

['title', 'description', 'price', 'salePercentage', 'recentReviews', 'allReviews']


In [ ]:
print(df.info)

<bound method DataFrame.info of                                                 title  \
0                       Ori and the Will of the Wisps   
1   Flashing Lights - Police, Firefighting, Emerge...   
2                                          Thronefall   
3   DRAGON QUEST® XI S: Echoes of an Elusive Age™ ...   
4                                             UNDYING   
..                                                ...   
81                         Bendy and the Dark Revival   
82  STAR WARS™ - The Force Unleashed™ Ultimate Sit...   
83                                           Thymesia   
84                                    Last Train Home   
85                                         Fallout 76   

                                          description   price salePercentage  \
0   Play the critically acclaimed masterpiece. Emb...   $9.89           -67%   
1   Play solo or in up to 10-player multiplayer co...   $8.49           -66%   
2   A minimalist game about building and de

In [ ]:
print(df.isnull().sum())

title              0
description        4
price             25
salePercentage    27
recentReviews     29
allReviews        29
dtype: int64


In [ ]:
print(df.describe())

                                title  \
count                              86   
unique                             75   
top     Ori and the Will of the Wisps   
freq                                2   

                                              description   price  \
count                                                  82      61   
unique                                                 71      34   
top     Play the critically acclaimed masterpiece. Emb...  $23.99   
freq                                                    2       6   

       salePercentage  recentReviews     allReviews  
count              59             57             57  
unique             17              5              5  
top              -40%  Very Positive  Very Positive  
freq               10             40             40  


In [ ]:
print(df['recentReviews'].value_counts())

recentReviews
Very Positive              40
Mostly Positive             8
Overwhelmingly Positive     7
Mixed                       1
Mostly Negative             1
Name: count, dtype: int64


## Data Cleaning


In [ ]:
print("before cleaning: ", df.shape)
df_clean = df.copy() # copy before cleaning

before cleaning:  (86, 6)


In [ ]:
def clean_price(p):
  if pd.isna(p) or not isinstance(p, str):
    return np.nan

  p = p.replace("$", "").strip()
  try:
    return float(p)
  except:
    return np.nan

# cleaning price from "$"
df['price_clean'] = df['price'].apply(clean_price)

def clean_sale(s):
  if pd.isna(s) or not isinstance(s, str):
    return 0
  s = s.replace("%", "").replace("-", "").strip()
  try:
    return float(s)
  except:
    return 0

# cleaning sale from "%"
df["sale_clean"] = df["salePercentage"].apply(clean_sale)

In [ ]:
# price after discount
df["effective_price"] = df["price_clean"] * (1-df["sale_clean"] / 100)

In [ ]:
# turning labels to numbers for analysis (to calc avgs/trends)
sentiment_map = {
    "Overwhelmingly Positive": 5,
    "Very Positive": 4,
    "Mostly Positive": 3,
    "Mixed": 2,
    "Mostly Negative": 1,
    "Very Negative": 0
}

df["recent_reviews_score"] = df["recentReviews"].map(sentiment_map)

In [ ]:
# creating price + discount buckets
df["price_bucket"] = pd.cut(df["price_clean"],
                            bins = [0, 5, 15, 30, 60, np.inf],
                            labels=["<$5", "$5-15", "$15-30", "$30-60", "$60+"])

df["discount_bucket"] = pd.cut(df["sale_clean"],
                               bins=[-1, 0, 25, 50, 75, 100],
                               labels=["No Sale", "1-25%", "25-50%", "50-75%", "75-100%+"])


In [ ]:
df_clean = df.dropna(subset=["price_clean", "recent_reviews_score"])

In [ ]:
df_clean.to_csv("steam_clean.csv", index=False)

```sql
-- Query to see if deeper discounts lead to higher review sentiment
SELECT
  discount_bucket,
  ROUND(AVG(recent_reviews_score), 2) AS avg_sentiment,
  COUNT(*) AS sample_size
FROM steam_clean
GROUP BY discount_bucket
ORDER BY
  CASE discount_bucket
    WHEN 'No Sale' THEN 1
    WHEN '1–25%' THEN 2
    WHEN '25–50%' THEN 3
    WHEN '50–75%' THEN 4
    WHEN '75–100%' THEN 5
  END;

```

```sql
-- Does the relationship between discounts and sentiment differ across price buckets?

SELECT
  price_bucket,
  discount_bucket,
  ROUND(AVG(recent_reviews_score), 2) AS avg_sentiment,
  COUNT(*) AS sample_size
FROM steam_clean
GROUP BY price_bucket, discount_bucket
ORDER BY
  CASE price_bucket
    WHEN '<$5'   THEN 1
    WHEN '$5-15' THEN 2
    WHEN '$15-30' THEN 3
    WHEN '$30-60' THEN 4
    WHEN '$60+'  THEN 5
  END,
  CASE discount_bucket
    WHEN 'No Sale' THEN 1
    WHEN '1–25%' THEN 2
    WHEN '25–50%' THEN 3
    WHEN '50–75%' THEN 4
    WHEN '75–100%' THEN 5
  END;


```
